# Web And Social Network Analytics - Final Project

### Task:
This code was used to determine the effectiveness of the three advertisement campaigns by calculating the most appropriate web analytics measures.

### Result:
The effectiveness of the three advertisement campaigns of HaggisBus was measured with different key points of interests (KPIs) and compared to common benchmarks in eCommerce. The average conversion rate (CR), here: the rate of users purchasing out of all visitors, is around 1.8% in the United Kingdom. The average bounce rate (BR), the rate of users leaving the website after seeing one page, and the average cart abandonment rate (CAR), the rate of people starting but not finishing the purchase process, are around 45-50% and 76.1%, respectively.

In [1]:
#1. get list with platforms (done in classes)
#2. get list with desktop and mobile

#if value not found, another value will be put in, similar to dictionary get() method, that doesn't exist for list
def safe_list_get (a_list, index, default_value):
    if index < len(a_list):
        return a_list[index]
    else:
        return default_value


class Visit:
    def __init__(self, line_of_file):
        line_of_file = line_of_file.rstrip("\n")
        data_items = line_of_file.split(",")
        self.source = safe_list_get(data_items,0,'unknown')
        self.platform = safe_list_get(data_items,1,'unknown')
        self.clickstream = data_items[2:]
        
    #for faulty data: @Q: is there a a simple way like with?
    
    #conversion
    def is_conversion(self):
        return 'purchase_success' in self.clickstream
    
    def abandon_cart(self):
        return 'purchase_start' in self.clickstream  and 'purchase_success' not in self.clickstream 
    
    def platform_is_desktop(self):
        return self.platform != "ios" and self.platform != "android" 
    
    def is_bouncer(self):
        return len(self.clickstream) <= 1 
    
    #get last page
    def exit_page(self):
        #print("cs", self.clickstream[len(self.clickstream)-1])
        return self.clickstream[len(self.clickstream)-1] 




In [2]:
def add_sum(visit_object,desktop_list, mobile_list,sumlist_Desktop_drop, sumlist_Mobile_drop):
    #for desktop
    if visit_object.platform_is_desktop():
        #@: Why can't I use global variables in a python function?
        desktop_list +=1
        #drop out after purchase_start (abandoned cart)    
        if visit_object.abandon_cart():
            sumlist_Desktop_drop +=1
    #for mobile
    elif not visit_object.platform_is_desktop():
        mobile_list +=1
        #drop out after purchase_start (abandoned cart)    
        if visit_object.abandon_cart():
            sumlist_Mobile_drop += 1

    return desktop_list, mobile_list,sumlist_Desktop_drop, sumlist_Mobile_drop



In [3]:
def add_to_dict(dic, key):
    if key in dic.keys():
        dic[key] += 1
    else:
        dic[key] = 1
    return dic

In [4]:
def add_item_to_list(data_item_object, given_list):
    given_list["users"] += 1
    given_list["last_pages"] = add_to_dict(given_list["last_pages"], data_item_object.clickstream[len(data_item_object.clickstream)-1] )
    if data_item_object.is_conversion():
        given_list["conversions"] += 1
    if data_item_object.is_bouncer():
        given_list["bounces"] += 1
            #new
        given_list["bounced_pages"] = add_to_dict(given_list["bounced_pages"], data_item_object.clickstream[len(data_item_object.clickstream)-1] )
            
    if data_item_object.abandon_cart():
        given_list["abandon_cart"] += 1   
            #new
        given_list["abandon_cart_pages"] = add_to_dict(given_list["abandon_cart_pages"], data_item_object.clickstream[len(data_item_object.clickstream)-1] )
        
    if "purchase_success" in data_item_object.clickstream:
        given_list["purchase_success"] += 1
    if "purchase_enter_address" in data_item_object.clickstream:
        given_list["purchase_enter_address"] += 1
    if "purchase_start" in data_item_object.clickstream:
        given_list["purchase_start"] += 1   
    
    return given_list
            

In [5]:
def add_item(data_item_object, facebook, linkedin, partner, l_s, fb_s, direct, search, mac, windows, ios, android, unknown, blog_1, blog_2, blog_1_2):
    #print(data_item_object.clickstream)
    if data_item_object.source == "facebook_advert":
        add_item_to_list(data_item_object, facebook)
    
    elif data_item_object.source == "linkedin_advert":
        add_item_to_list(data_item_object, linkedin)
        
    elif data_item_object.source == "partner_advert":
        add_item_to_list(data_item_object, partner)
        
    elif data_item_object.source == "linkedin_share":
        add_item_to_list(data_item_object, l_s)
    
    elif data_item_object.source == "facebook_share":
        add_item_to_list(data_item_object, fb_s)
        
    elif data_item_object.source == "direct":
        add_item_to_list(data_item_object, direct)

    elif data_item_object.source == "search":
        add_item_to_list(data_item_object, search)
    
    if data_item_object.platform == "mac":
        add_item_to_list(data_item_object, mac)
    elif data_item_object.platform == "windows":
        add_item_to_list(data_item_object, windows)
    elif data_item_object.platform == "ios":
        add_item_to_list(data_item_object, ios)
    elif data_item_object.platform == "android":
        add_item_to_list(data_item_object, android)
    else:
        add_item_to_list(data_item_object, unknown)
        
    
    if "blog_1" in data_item_object.clickstream and "blog_2" in data_item_object.clickstream:
        add_item_to_list(data_item_object, blog_1_2)
    elif "blog_1" in data_item_object.clickstream:
        add_item_to_list(data_item_object, blog_1)
    elif "blog_2" in data_item_object.clickstream:
        add_item_to_list(data_item_object, blog_2)
        
    
    return facebook, linkedin, partner, l_s, fb_s, direct, search, mac, windows, ios, android, unknown, blog_1, blog_2, blog_1_2
    
    
    

In [6]:
mac = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

windows = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

ios = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

android = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

unknown = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

In [7]:
l_s = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

fb_s = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

direct = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

search = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}


In [8]:
blog_1 = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

blog_2 = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}

blog_1_2 = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}


In [9]:
facebook = {"users" : 0,
                "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}
linkedin = {"users" : 0,
            "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0}
partner = {"users" : 0,
            "conversions" : 0,
                "bounces" : 0,
               "last_pages": {},
               "abandon_cart" : 0,
           "bounced_pages": {},
           "abandon_cart_pages": {},
          "purchase_start": 0,
          "purchase_enter_address": 0,
          "purchase_success": 0} 

In [10]:
with open('visitor_data_clickstream.csv') as file:
    line = file.readline() #ignore first line

    
    while line:
        data_item_object = Visit(line)
        #print("while loop")
        facebook, linkedin, partner, l_s, fb_s, direct, search, mac, windows, ios, android, unknown, blog_1, blog_2, blog_1_2 = add_item(data_item_object, facebook, linkedin, partner, l_s, fb_s, direct, search, mac, windows, ios, android, unknown, blog_1, blog_2, blog_1_2)
        line = file.readline()
        
        
        



## Task 1

In [11]:
print("FB Conversion rate: ", facebook["conversions"]/facebook["users"])
print("FB Bounce rate: ", facebook["bounces"]/facebook["users"])
print("FB now correct abandon_cart rate: ", ((1-(facebook["conversions"]/facebook["purchase_start"]))))
print("FB highest exit pages: ", facebook["last_pages"])
print("FB most bounced pages: ", facebook["bounced_pages"])
print("/nL rate:", linkedin)
print("L Conversion rate: ", linkedin["conversions"]/linkedin["users"])
print("L Bounce rate: ", linkedin["bounces"]/linkedin["users"])
print("L now correct abandon_cart rate: ", ((1-(linkedin["conversions"]/linkedin["purchase_start"]))))
print("L highest exit pages: ", linkedin["last_pages"])
print("L most bounced pages: ", linkedin["bounced_pages"])
print("/npartner rate:", partner)
print("Partner Conversion rate: ", partner["conversions"]/partner["users"])
print("Partner Bounce rate: ", partner["bounces"]/partner["users"])
print("Partner abandon_cart rate: ", ((1-(partner["conversions"]/partner["purchase_start"]))))
print("Partner highest exit pages: ", partner["last_pages"])
print("Partner most bounced pages: ", partner["bounced_pages"])

FB Conversion rate:  0.0093
FB Bounce rate:  0.4262
FB now correct abandon_cart rate:  0.967032967032967
FB highest exit pages:  {'purchase_start': 1556, 'special_advert_deal': 4262, 'contact_us': 637, 'purchase_enter_address': 756, 'pricing': 2348, 'purchase_success': 83, 'home': 154, 'blog_2': 64, 'blog_1': 140}
FB most bounced pages:  {'special_advert_deal': 4262}
/nL rate: {'users': 2000, 'conversions': 504, 'bounces': 0, 'last_pages': {'contact_us': 462, 'blog_1': 128, 'purchase_success': 376, 'pricing': 355, 'purchase_enter_address': 224, 'purchase_start': 291, 'home': 107, 'blog_2': 57}, 'abandon_cart': 896, 'bounced_pages': {}, 'abandon_cart_pages': {'contact_us': 191, 'purchase_enter_address': 223, 'purchase_start': 287, 'pricing': 80, 'blog_1': 45, 'blog_2': 13, 'home': 57}, 'purchase_start': 1400, 'purchase_enter_address': 1099, 'purchase_success': 504}
L Conversion rate:  0.252
L Bounce rate:  0.0
L now correct abandon_cart rate:  0.64
L highest exit pages:  {'contact_us': 

## Task 2

In [12]:
def print_out(item):
    print(item["conversions"]/item["users"])
    print(item["bounces"]/item["users"])
    print((1-(item["conversions"]/item["purchase_start"])))
    print(item["last_pages"])
    print(item["bounced_pages"])
    print(item["abandon_cart_pages"])
    print(item["purchase_start"])
    print(item["purchase_enter_address"])
    print(item["purchase_success"])

In [13]:
print("\nlist of Mac")
print_out(mac)
print("\nlist of windows")
print_out(windows)
print("\nlist of ios")
print_out(ios)
print("\nlist of android")
print_out(android)
print("\nlist of unknown")
print_out(unknown)


list of Mac
0.16003571428571428
0.16060714285714286
0.5842071077294237
{'contact_us': 6194, 'blog_2': 704, 'blog_1': 3686, 'pricing': 4150, 'purchase_success': 3730, 'purchase_enter_address': 1232, 'purchase_start': 2562, 'home': 4901, 'special_advert_deal': 841}
{'contact_us': 831, 'special_advert_deal': 841, 'blog_1': 880, 'home': 1863, 'blog_2': 82}
{'purchase_enter_address': 1229, 'contact_us': 1104, 'purchase_start': 2529, 'blog_1': 295, 'pricing': 665, 'home': 444, 'blog_2': 30}
10777
7497
4481

list of windows
0.164
0.16457142857142856
0.5756007393715342
{'blog_1': 3626, 'home': 4948, 'contact_us': 6231, 'purchase_start': 2513, 'purchase_success': 3802, 'pricing': 4094, 'blog_2': 708, 'special_advert_deal': 854, 'purchase_enter_address': 1224}
{'home': 1955, 'blog_2': 91, 'special_advert_deal': 854, 'blog_1': 880, 'contact_us': 828}
{'purchase_start': 2490, 'contact_us': 1064, 'pricing': 679, 'purchase_enter_address': 1219, 'home': 428, 'blog_1': 325, 'blog_2': 23}
10820
7584
4

## Task 3

In [14]:
print("\nlist of l_s")
print_out(l_s)
print("\nlist of fb_s")
print_out(fb_s)
print("\nlist of direct")
print_out(direct)
print("\nlist of search")
print_out(search)


list of l_s
0.12009433962264152
0.19995283018867924
0.6586673816865531
{'purchase_success': 1802, 'home': 3628, 'contact_us': 5165, 'blog_2': 658, 'pricing': 2881, 'purchase_enter_address': 1413, 'blog_1': 4136, 'purchase_start': 1517}
{'home': 915, 'blog_2': 172, 'blog_1': 2167, 'contact_us': 985}
{'contact_us': 867, 'purchase_enter_address': 1406, 'purchase_start': 1496, 'pricing': 514, 'home': 365, 'blog_1': 243, 'blog_2': 22}
7459
5636
2546

list of fb_s
0.12976608187134503
0.15838206627680312
0.6496684559520051
{'blog_1': 10212, 'contact_us': 10813, 'pricing': 7183, 'purchase_success': 4683, 'purchase_enter_address': 3429, 'home': 9304, 'purchase_start': 3919, 'blog_2': 1757}
{'home': 2791, 'blog_1': 4874, 'blog_2': 460}
{'blog_1': 636, 'purchase_enter_address': 3417, 'contact_us': 2193, 'purchase_start': 3854, 'pricing': 1354, 'home': 834, 'blog_2': 57}
19002
14252
6657

list of direct
0.31666666666666665
0.09533333333333334
0.2178924259055982
{'contact_us': 2397, 'home': 2533, 

## Task 4

In [15]:
print("\nlist of blog_1")
print_out(blog_1)
print("\nlist of blog_2")
print_out(blog_2)
print("\nlist of blog_1_2")
print_out(blog_1_2)


list of blog_1
0.09323432343234324
0.15924092409240925
0.6504124926340601
{'purchase_success': 3399, 'blog_1': 22035, 'contact_us': 8195, 'pricing': 4604, 'home': 8190, 'purchase_enter_address': 2027, 'purchase_start': 2454}
{'blog_1': 8106}
{'blog_1': 1374, 'purchase_enter_address': 2021, 'contact_us': 1370, 'purchase_start': 2428, 'pricing': 957, 'home': 680}
13576
9795
4746

list of blog_2
0.1794707849695062
0.03565807593940586
0.5836376083979917
{'purchase_start': 1783, 'purchase_success': 2478, 'blog_2': 3623, 'purchase_enter_address': 1537, 'contact_us': 3783, 'pricing': 3859, 'home': 3269}
{'blog_2': 725}
{'purchase_start': 1738, 'purchase_enter_address': 1524, 'pricing': 554, 'contact_us': 861, 'home': 329, 'blog_2': 109}
8764
6582
3649

list of blog_1_2
0.1639376996805112
0.0
0.5949679329057721
{'blog_1': 1356, 'pricing': 890, 'contact_us': 599, 'purchase_success': 272, 'blog_2': 600, 'purchase_enter_address': 164, 'purchase_start': 255, 'home': 872}
{}
{'contact_us': 153, 'p